In [29]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import sleep
import tkinter as tkconda 
from tkinter import ttk 
import serial.tools.list_ports
import serial
from datetime import datetime
import pandas as pd
from itertools import combinations
import keyboard
from model_2 import ConvNet

dim = 5
len_of_window_to_preprocessing = 10
len_of_trial = 8
classes = ('background', 'bottle', 'box', 'powerbank', 'cucumber', 'banana')

ser = serial.Serial('COM3', 9600)
ser.flushInput()

def compute_matrix_of_means(dim, len_of_window):
    data_array_for_compute_means = np.zeros((dim, dim, len_of_window), dtype="int64")
    for step_of_full_fill in range (len_of_window):
        data_array = np.zeros((dim, dim), dtype="int64")
        for rows in range(dim):
            data = ser.readline().decode('utf-8').strip()
            values = np.array([int(x) for x in data.split()])
            match values[0]:
                case 0: data_array[0] = values[1:]
                case 1: data_array[1] = values[1:]
                case 2: data_array[2] = values[1:]
                case 3: data_array[3] = values[1:]
                case 4: data_array[4] = values[1:]
        data_array_for_compute_means[:, :, step_of_full_fill] = data_array
    return np.mean(data_array_for_compute_means, axis=2) 
        
def add_new(a, b):
  a = np.delete(a, 0, axis=0)
  a = np.vstack((a, b))
  return a

def pairwise_subtractions(arr):
  valid_values = np.array(list((combinations(arr, 2))))
  values = valid_values[:, 0] - valid_values[:, 1]
  return values

def compute_matrix_of_relationships(data):
  new_full_matrix =  np.zeros((int(data.shape[0] * (data.shape[1] - 1) / 2), int(data.shape[0] * (data.shape[1] - 1) / 2)), dtype="float64")
  new_only_rows_matrix = np.zeros((data.shape[0], int(data.shape[0] * (data.shape[1] - 1) / 2)), dtype="float64")
  for j in range(int((data.shape[0] * (data.shape[1] - 1)) / 2)):
      for i in range(data.shape[0]):
          new_only_rows_matrix[i, :] = pairwise_subtractions(data[i, :])
      new_full_matrix[:, j] = pairwise_subtractions(new_only_rows_matrix[:, j])
  return new_full_matrix



def main_loop():
    ser.flushInput()
    with open(txt_file, "w") as txtfile:
        while True:
            ser.flushInput() 
            # Check for 'q' key press to break the loop
            if keyboard.is_pressed('q'):
                print('q')
                break
            one_sample_data = np.zeros(( len_of_trial, 25, 12), dtype="float64")
            
            for trial in range(len_of_trial):
                one_matrix = np.zeros((5, 5))
                for rows in range(dim):
                    data = ser.readline().decode('utf-8').strip()
                    values = np.array([int(x) for x in data.split()])
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')
                    txtfile.write(f"{values},{timestamp}\n")
                    if len(values) == 6:
                        one_matrix[values[0]] = values[1:]
                one_matrix -= matrix_of_means
                
                one_matrix = pairwise_subtractions(one_matrix.reshape(25)).reshape(25, 12)
                
                one_sample_data[trial, :, :] = one_matrix

            one_sample_data = torch.Tensor(one_sample_data)
            outputs = model(one_sample_data)
            _, predicted = torch.max(outputs, 1)
            print(classes[predicted])

    
    

              
if __name__ == '__main__':
    txt_file = 'data_prediction.txt'
    model = torch.load('cnn (1).pth', weights_only=False, map_location=torch.device('cpu'))
    matrix_of_means = compute_matrix_of_means(dim, len_of_window_to_preprocessing)

    print("Start")
    
    main_loop()

    
 

ImportError: cannot import name 'ConvNet' from 'model_2' (d:\Visual studio\projects\glove\model_2.py)